## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

user_question = "I just discovered the course. Can I still join it?"
user_question_encoded = embedding_model.encode(user_question)
round(user_question_encoded[0], 3)

/home/codespace/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


0.078

### What's the first value of the resulting vector? 
- 0.078

### Prepare the documents

In [2]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
use_docs = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]
use_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [4]:
len(use_docs)

375

## Q2. Creating the embeddings

In [5]:
import numpy as np
embeddings = []
for doc in use_docs:
    question = doc['question']
    text=doc['text']
    qa_text = f'{question} {text}'
    qa_text_encoded = embedding_model.encode(qa_text)
    embeddings.append(qa_text_encoded)

X = np.array(embeddings)
X.shape

(375, 768)

### What's the shape of X?
- (375, 768)

## Q3. Search

In [6]:
user_question_encoded.shape

(768,)

In [7]:
v = user_question_encoded
scores = X.dot(v)

In [8]:
scores.max()

0.6506573

### What's the highest score in the results?
- 0.6506573

### Vector search

In [44]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

## Q4. Hit-rate for our search engine

In [45]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [46]:
search_engine = VectorSearchEngine(documents=use_docs, embeddings=X)

doc_match = []
for doc in ground_truth:
    question = doc['question']
    v = embedding_model.encode(question)
    result = search_engine.search(v, num_results=5)
    match = [doc['document'] == res['id'] for res in result]
    doc_match.append(match)

In [49]:
hit_sum = 0
for i in doc_match:
    hit_sum += sum(i)
hit_rate = hit_sum/len(doc_match)

hit_rate

0.9426229508196722

### What did you get?
- 0.9426

## Q5. Indexing with Elasticsearch

In [52]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

In [60]:
use_docs[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id'])

In [ ]:
for i in range(len(use_docs)):
    use_docs[i]["text_vector"] =  X[i]

In [65]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            'id': {"type": "text"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [68]:
index_name = "ml_questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml_questions'}

In [69]:
for doc in use_docs:
    try:
        es_client.index(index=index_name, body=doc)
    except Exception as e:
        print(e)

In [70]:
query = {
    "knn": {
        "field": "text_vector",
        "query_vector": user_question_encoded,
        "k": 5, 
        "num_candidates": 10000
        },
    
    "fields": ["text","section","question","course", "id"]
}

In [ ]:
results = es_client.search(index=index_name, body=query)
results["hits"]["hits"]

### What's the ID of the document with the highest score?
- ee58a693

## Q6. Hit-rate for Elasticsearch

In [90]:
def search_es(search_vector):
    query = {
    "knn": {
        "field": "text_vector",
        "query_vector": search_vector,
        "k": 5, 
        "num_candidates": 10000
        },
    
    "fields": ["text","section","question","course", "id"]
    }
    
    results = es_client.search(index=index_name, body=query)
    return results["hits"]["hits"]

In [91]:
from tqdm import tqdm

doc_match = []
for doc in tqdm(ground_truth):
    question = doc['question']
    v = embedding_model.encode(question)
    result = search_es(v)
    match = [doc['document'] == res['_source']["id"] for res in result]
    doc_match.append(match)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:24<00:00, 21.60it/s]


In [92]:
hit_sum = 0
for i in doc_match:
    hit_sum += sum(i)
hit_rate = hit_sum/len(doc_match)

hit_rate

0.9426229508196722

What's hitrate for our dataset for Elastic?
- 0.9426